In [3]:
# This file aims to display some functions regarding Brightway 2.5, to demonstrate what can be done with the software when you have a foreground model matched with ecoinvent's bio- and technosphere.

# Analysis

### 1. Loading the project

In [4]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns

In [5]:
# define a project where we install the databases and work in this script
bd.projects.set_current('brightway25')

### 2. Database overview

In [6]:
bd.databases

Databases dictionary with 4 object(s):
	biosphere3
	bw25_db
	ecoinvent-3.10-biosphere
	ecoinvent310clca

In [7]:
bd.projects.current

'brightway25'

In [8]:
ei_clca = bd.Database('ecoinvent310clca')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')
db_template = bd.Database('bw25_db')
el_bio3 = bd.Database('biosphere3')

### 3. Database exploration

In [9]:
db_template.search('ceramics')

['porcelain ceramics production' (kilogram, RER, None)]

In [10]:
ei_clca.search('ceramics')

['market for sanitary ceramics' (kilogram, GLO, None),
 'sanitary ceramics production' (kilogram, RoW, None),
 'sanitary ceramics production' (kilogram, CH, None),
 'refractory production, basic, packed' (kilogram, RoW, None),
 'refractory production, fireclay, packed' (kilogram, DE, None),
 'refractory production, fireclay, packed' (kilogram, RoW, None),
 'refractory production, basic, packed' (kilogram, DE, None),
 'market for ceramic tile' (kilogram, GLO, None),
 'ceramic tile production' (kilogram, CH, None),
 'market for feldspar' (kilogram, GLO, None),
 'power saw production, with catalytic converter' (unit, GLO, None),
 'ceramic tile production' (kilogram, RoW, None),
 'market for titanium dioxide' (kilogram, RoW, None),
 'market for titanium dioxide' (kilogram, RER, None),
 'market for kaolin' (kilogram, GLO, None),
 'strontium carbonate production' (kilogram, GLO, None),
 'sodium tripolyphosphate production' (kilogram, RoW, None),
 'sodium tripolyphosphate production' (kilogra

In [11]:
# define a process as object in the project
ceramics_bw = db_template.get(name = 'porcelain ceramics production', location = "RER", unit = 'kilogram')
ceramics_ei = ei_clca.get(name = 'sanitary ceramics production', location = "CH", unit = 'kilogram')


In [12]:
# quickly print all exchanges of the process defined above, sorted by amount (large to small)
exchanges = sorted(ceramics_bw.exchanges(), key=lambda exc: exc['amount'], reverse=True)
for exc in exchanges:
    print(exc)

Exchange: 1.0277 kilowatt hour 'market group for electricity, high voltage' (kilowatt hour, RER, None) to 'porcelain ceramics production' (kilogram, RER, None)>
Exchange: 1 kilogram 'porcelain ceramics production' (kilogram, RER, None) to 'porcelain ceramics production' (kilogram, RER, None)>
Exchange: 0.5775000000000001 kilogram 'market for kaolin' (kilogram, GLO, None) to 'porcelain ceramics production' (kilogram, RER, None)>
Exchange: 0.45066666666666666 cubic meter 'market group for natural gas, high pressure' (cubic meter, Europe without Switzerland, None) to 'porcelain ceramics production' (kilogram, RER, None)>
Exchange: 0.275 kilogram 'market for feldspar' (kilogram, GLO, None) to 'porcelain ceramics production' (kilogram, RER, None)>
Exchange: 0.24750000000000003 kilogram 'market for silica sand' (kilogram, GLO, None) to 'porcelain ceramics production' (kilogram, RER, None)>
Exchange: 0.01261 kilogram 'Sulfur dioxide' (kilogram, None, ('air',)) to 'porcelain ceramics productio

### 4. Impact assessment methods

In [13]:
# here we will analyse the set of methods that are available as part of the background
list(bd.methods)[:5]

[('CML v4.8 2016 no LT',
  'acidification no LT',
  'acidification (incl. fate, average Europe total, A&B) no LT'),
 ('CML v4.8 2016 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: freshwater no LT',
  'freshwater aquatic ecotoxicity (FAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: marine no LT',
  'marine aquatic ecotoxicity (MAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: terrestrial no LT',
  'terrestrial ecotoxicity (TETP inf) no LT')]

In [14]:
# printing all methods that contain a specific keyword, e.g. "climate change"
climate_methods = [m for m in bd.methods if any("climate change" in str(part).lower() for part in m)]
for method in climate_methods:
    print(method)

('CML v4.8 2016 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')
('Ecological Scarcity 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
('Ecological Scarcity 2021', 'climate change', 'global warming potential (GWP100)')
('EF v3.0 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
('EF v3.0 no LT', 'climate change: biogenic no LT', 'global warming potential (GWP100) no LT')
('EF v3.0 no LT', 'climate change: fossil no LT', 'global warming potential (GWP100) no LT')
('EF v3.0 no LT', 'climate change: land use and land use change no LT', 'global warming potential (GWP100) no LT')
('EF v3.0', 'climate change', 'global warming potential (GWP100)')
('EF v3.0', 'climate change: biogenic', 'global warming potential (GWP100)')
('EF v3.0', 'climate change: fossil', 'global warming potential (GWP100)')
('EF v3.0', 'climate change: land use and l

In [15]:
# selecting the impact assessment methods that we want to use in our analysis
lcia_gwp100 = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')
lcia_water = ('EF v3.1','water use','user deprivation potential (deprivation-weighted water consumption)')
lcia_land = ('EF v3.1', 'land use', 'soil quality index')

### 5. LCI 

In [16]:
# export the LCI to excel in an overview format
bi.export.excel.write_lci_excel(db_template.name,
                                objs=[ceramics_bw],
                                dirpath=Path.cwd())

'c:\\Users\\TimWeber\\repos_20LCA\\LCA_Toolbox\\lci-bw25_db.xlsx'

### 6. LCIA

In [17]:
# Quick LCIA calculation
ceramics_bw_lca = ceramics_bw.lca(lcia_gwp100)
ceramics_bw_lca.score


0.7265791393172021

In [18]:
ceramics_ei_lca = ceramics_ei.lca(lcia_gwp100)
ceramics_ei_lca.score

3.204853545579472

### 7. Contribution analysis

In [ ]:
# One option to do a contribution analysis is to use the recursive calculation method
bwa.print_recursive_calculation(ceramics_bw,
lcia_method=lcia_gwp100,max_level=3,cutoff=0.05)

Fraction of score | Absolute score | Amount | Activity
0001 | 0.7266 |     1 | 'porcelain ceramics production' (kilogram, RER, None)
  00.2 | 0.1451 | 0.5775 | 'market for kaolin' (kilogram, GLO, None)
    0.123 | 0.08954 | 0.3783 | 'kaolin production' (kilogram, RoW, None)
      0.0546 | 0.0397 | 0.05354 | 'market group for electricity, medium voltage' (kilowatt hour, RAS, None)
    0.0699 | 0.05079 | 0.1992 | 'kaolin production' (kilogram, RER, None)
  0.331 | 0.2407 | 1.028 | 'market group for electricity, high voltage' (kilowatt hour, RER, None)
    0.327 | 0.2376 | 1.013 | 'market group for electricity, high voltage' (kilowatt hour, Europe without Switzerland, None)
      0.124 | 0.08975 | 0.1495 | 'market for electricity, high voltage' (kilowatt hour, UA, None)
  0.389 | 0.283 | 0.4507 | 'market group for natural gas, high pressure' (cubic meter, Europe without Switzerland, None)
    0.0665 | 0.04834 | 0.06835 | 'market for natural gas, high pressure' (cubic meter, IT, None)
    

In [ ]:
# Another option is to use the recursive calculation to an object, which returns a DataFrame
ceramics_bw_ca = bwa.utils.recursive_calculation_to_object(ceramics_bw,
                                          lcia_method=lcia_gwp100,
                                          max_level=1,
                                          cutoff=0.02,
                                          as_dataframe=True,
                                          )
ceramics_bw_ca

In [ ]:
# The goal here is to conduct the same analysis for the ecoinvent process so we can compare the contributions later
ceramics_ei_ca = bwa.utils.recursive_calculation_to_object(ceramics_ei,
                                          lcia_method=lcia_gwp100,
                                          max_level=1,
                                          cutoff=0.02,
                                          as_dataframe=True,
                                          )
ceramics_ei_ca

In [ ]:
# Elementary flows contribution analysis by process
ceramics_ei_lca.to_dataframe().pivot_table(index=['col_name', 'row_name'], values='amount', aggfunc='sum').sort_values(by='amount', ascending=False)

In [ ]:
# Elementary flows contribution analysis as summary table
ceramics_ei_lca.to_dataframe().pivot_table(index='row_name',values='amount',aggfunc='sum')

### 8. Plot Contribution analysis graphs

In [ ]:
# Filter out the parent processes that are not relevant for the contribution analysis
ceramics_bw_ca = ceramics_bw_ca.dropna(subset='parent') 
ceramics_ei_ca = ceramics_ei_ca.dropna(subset='parent')



In [ ]:
# First, we plot the contribution analysis for the brigthway ceramics production process
f, ax = plt.subplots(figsize=(6, 3))
sns.barplot(y='name', x='score', data=ceramics_bw_ca, ax=ax)
ax.set(
    title='Contribution analysis of the product under study',
    xlabel='CO2 eq. emissions (kg)',
    ylabel='')
plt.tight_layout()
plt.show()


In [ ]:
# Then, we plot the same for the ecoinvent ceramics production process
f, ax = plt.subplots(figsize=(6, 3))
sns.barplot(y='name', x='score', data=ceramics_ei_ca, ax=ax)
ax.set(
    title='Contribution analysis of the product under study',
    xlabel='CO2 eq. emissions (kg)',
    ylabel='')
plt.tight_layout()
plt.show()


### 9. Comparison

In [ ]:
combined_df = pd.concat([ceramics_bw_ca.set_index('name').score,ceramics_ei_ca.set_index('name').score],
                        keys=['ceramics_bw','ceramics_ei'],
                        names=['activity','contributor'])
combined_df

In [ ]:
ax = combined_df.unstack().plot.barh(
    stacked=True,
    title='Main contributors to global warming',
    xlabel='kg CO2eq',
    figsize=(10,6)
)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # moves legend outside
plt.tight_layout()
plt.show()